<a href="https://www.kaggle.com/code/iqmansingh/classification-titanic-dataset-baseline?scriptVersionId=133682877" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
sns.set_style("dark") 
sns.color_palette("flare")
warnings.filterwarnings('ignore')
print("Setup Complete")

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv
Setup Complete


In [2]:
titanic_file_path = '/kaggle/input/titanic/train.csv'
titanic_test_path = '/kaggle/input/titanic/test.csv'

titanic_data = pd.read_csv(titanic_file_path)
titanic_test = pd.read_csv(titanic_test_path)

titanic_data.drop(["Name"],inplace=True,axis=1)
titanic_test.drop(["Name"],inplace=True,axis=1)
titanic_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
print(titanic_data.info(),"\n\n\n",titanic_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          714 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Sex     

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


titanic_data["Embarked"].fillna(method="ffill",inplace=True)
titanic_data["Age"].interpolate(inplace=True)
titanic_data.drop(["Cabin","Ticket"],axis=1,inplace=True)

titanic_test["Embarked"].fillna(method="ffill",inplace=True)
titanic_test["Age"].interpolate(inplace=True)
titanic_test["Fare"].interpolate(inplace=True)
titanic_test.drop(["Cabin","Ticket"],axis=1,inplace=True)

encoder = LabelEncoder()

titanic_data["Sex"] = encoder.fit_transform(titanic_data["Sex"])
titanic_test["Sex"] = encoder.transform(titanic_test["Sex"])

titanic_data["Embarked"] = encoder.fit_transform(titanic_data["Embarked"])
titanic_test["Embarked"] = encoder.transform(titanic_test["Embarked"])

In [5]:
titanic_data["Embarked"].unique() 

array([2, 0, 1])

In [6]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier

In [7]:
cols = [i for i in titanic_data.columns if i!="Survived"]
seed = np.random.seed(0)

X_train, X_test, y_train, y_test = train_test_split(titanic_data[cols], titanic_data["Survived"], test_size=0.3,random_state=seed)

In [8]:
# train_x = titanic_data[cols]  # Full training data
# train_y = titanic_data["Survived"] # Full training data

train_x = X_train
train_y = y_train

In [9]:
rfmodel = RandomForestClassifier(random_state=seed)
rfmodel.fit(train_x,train_y)
print("Accuracy of RF:",accuracy_score(y_test, rfmodel.predict(X_test))*100)

Accuracy of RF: 79.8507462686567


In [10]:
gbmodel = GradientBoostingClassifier(random_state=seed)
gbmodel.fit(train_x,train_y)
print("Accuracy of GB:",accuracy_score(y_test, gbmodel.predict(X_test))*100)

Accuracy of GB: 81.71641791044776


In [11]:
hgbmodel = HistGradientBoostingClassifier(random_state=seed)
hgbmodel.fit(train_x,train_y)
print("Accuracy of HGB:",accuracy_score(y_test, hgbmodel.predict(X_test))*100)

Accuracy of HGB: 80.59701492537313


In [12]:
xgbmodel = XGBClassifier(random_state=seed)
xgbmodel.fit(train_x,train_y)
print("Accuracy of XGB:",accuracy_score(y_test, xgbmodel.predict(X_test))*100)

Accuracy of XGB: 80.59701492537313


In [13]:
lgbmmodel = LGBMClassifier(random_state=seed)
lgbmmodel.fit(train_x,train_y)
print("Accuracy of LGBM:",accuracy_score(y_test, lgbmmodel.predict(X_test))*100)

Accuracy of LGBM: 80.59701492537313


In [14]:
catmodel = CatBoostClassifier(random_state=seed,verbose=False)
catmodel.fit(train_x,train_y)
print("Accuracy of CAT:",accuracy_score(y_test, catmodel.predict(X_test))*100)

Accuracy of CAT: 81.71641791044776


In [15]:
vcmodel = VotingClassifier([("rf",rfmodel),("hgb",hgbmodel),("gb",gbmodel),("cat",catmodel)])
vcmodel.fit(train_x,train_y)
print("Accuracy of VC:",accuracy_score(y_test, vcmodel.predict(X_test))*100)

Accuracy of VC: 83.2089552238806


In [16]:
models = {"rf":rfmodel,"gb":gbmodel,"hgb":hgbmodel,"xgb":xgbmodel,"lgbm":lgbmmodel,"cat":catmodel,"vc":vcmodel}

In [17]:
for j in models.keys():
    submissions = titanic_test.copy()
    prediction = models[j].predict(titanic_test)
    
    colsToDrop = [i for i in titanic_test.columns if i!="PassengerId"]
    
    submissions.drop(colsToDrop,inplace=True,axis=1)
    submissions["Survived"] = prediction
    
    name = f"submission_{j}.csv"
    submissions.to_csv(name,header=True,index=False)    